In [87]:
import sqlite3
con = sqlite3.connect('db')

In [88]:
import pandas as pd

In [89]:
sqlite3.sqlite_version

'3.39.4'

In [90]:
df = pd.read_csv('database/german_credit_augmented.csv')
df.head()

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,default,contract_dt,client_id
0,33,male,2,own,NaN,NaN,3074,9,radio/TV,0,2008-06-29 18:52:00,210
1,43,male,1,own,little,little,1344,12,car,0,2007-05-20 18:30:19,929
2,52,male,2,own,quite rich,NaN,936,9,education,0,2008-04-27 08:23:07,200
3,35,female,3,own,little,NaN,1393,11,car,0,2007-05-06 10:58:22,45
4,28,male,2,own,little,NaN,776,12,radio/TV,0,2007-07-21 13:22:14,358


In [91]:
df.to_sql('german_credit', con, index=False, if_exists='replace')

1000

In [92]:
def select(sql):
    return pd.read_sql(sql, con)

In [93]:
sql = '''select t.sex,
count(1)
from german_credit t
group by t.sex'''

In [94]:
select(sql)

,sex,count(1)
0,female,310
1,male,690


In [95]:
df.groupby('sex')['credit_amount'].agg(['count', 'mean'])

,count,mean
sex,,
female,310,2877.774194
male,690,3448.040580


In [96]:
sql = '''select t.sex,
count(1) as cnt,
avg(t.credit_amount) as credit_amount_avg
from german_credit t
group by t.sex'''

In [97]:
select(sql)

,sex,cnt,credit_amount_avg
0,female,310,2877.774194
1,male,690,3448.040580


In [98]:
sql = '''select distinct t.housing from german_credit t'''

In [99]:
select(sql)

,housing
0,own
1,rent
2,free
